## Workshop "Неидеальный код"

Настало время засучить рукава и написать настоящее Spark приложение, а не сидеть в этом вашем юпитере. Вам предстоит работать с уже привычным датасетом ```nbagames.json```, но на этот раз результат должен быть оформлен в виде двух файлов:
+ spark-submit скрипт ```run-job.sh```
+ ваш код ```app.py```

### Условия
Представьте, что вы переделываете с нуля приложение, которое до вас написал неопытный Spak инженер. Основной задачей приложения является выполнение ETL и построение ТОП-10 отчета по данным. При этом вам периодически приходят новые данные в формате, соответствующему строкам файла ```nbagames.json```. Неопытный инженер просто сохранял новые данные в оригинальном формате, что делало построение ТОП-10 отчета очень медленным (помимо его кривого кода). В отличие от него, вы решили сохранять данные в иной, оптимальной структуре. Для этого вы решили использовать наиболее распространенный parquet формат. 

Вам необходимо определить, в какой структуре будут сохранятся новые данные. Например, вы можете сразу "взрывать" колонку "teams", использовать определенные колонки для партиционирования, или даже сохранять новые данные в несколько разных файлов. Важно одно - на момент приема новых данных у вас нет возможности перестраивать агрегаты по всему датасету, т.к. новые данные могут приходить достаточно часто. 

Ваше приложение должно работать в двух режимах (выбор режима определяется аргументом в командной строке).

Первый режим должен брать все новые данные из папки X и дописывать их в один или несколько parquet файлов в нужной структуре, расположенных в папке Y. X и Y используйте любые удобные вам.

Второй режим запускает генерацию ТОП-10 отчета из данных, расположенных в каталоге Y. Алгоритм уже реализован вашим предшественником и представлен ниже в этом ноутбуке. Вам необходимо адаптировать код под новый формат хранения данных в parquet и переписать код, чтобы он работал как можно быстрее, т.к. заказчик просит обновлять данный отчет как можно чаще.

К сожалению, перед увольнением инженер решил испортить код и теперь он не работает из-за внесенных синтаксических ошибок

P.S. ```run-job.sh``` должен запускаться из консоли и использовать YARN в качестве менеджера ресурсов для Spark.

Подсказки:
+ Найдите синтаксические ошибки в коде
+ Запустите код и посмотрите что он возвращает
+ Проанализируйте код и подумайте, какие данные и в каком виде вам нужны для построения отчета
+ Не вдавайтесь в смысл отчета - его там не очень много :)
+ не пытайтесь повторить все, что написано ниже в коде. Многие вещи там не нужны и будут сбивать вас с толку. Опирайтесь на поставленную задачу.

In [ ]:
input_dir = "hdfs:///user/atitov/nbagames.json"
parquet_db = "useless-parquet-file"

In [ ]:
import json
rdd = sc.textFile(input_dir)
var0 = rdd.repartition(1).map(lambda x: json.loads(x)['_id']['$oid']).cache.collect()

var1 = rdd.repartition(1).flatMap(lambda x: json.loads(x)['teams']).collect()

new_rdd = sc.parallelize(zip(var0, var1))

def flatten_json(x):
    a = x[1]
    a.update({'game_id': x[0]})
    return json.dumps(a)

new_rdd = new_rdd.map(lambda x: flatten_json(x))

df = spark.read.json(new_rdd)

from pyspark.sql.functions import * 

players = \
    df.select(col("abbreviation").alias("abb"), explode(col("players"))) \
    .groupBy(col("player.player").alias("player")).agg(collect_set(col("abb")).alias("abb"), count("*").alias("count"))

players.coalesce(1).write.mode("overwrite").partitionBy("count").parquet(parquet_db)

from pyspark.sql.types import *
from pyspark.sql.functions import udf, expr

players = spark.read.parquet(parquet_db)

join_condition = udf(lambda x,y: x in y, BooleanType)
result = df.join(players, join_condition(col("abbreviation"), col("abb"))) \
    .filter(col("player").startswith("Anthony"))

result.select(col("player"), col("abbreviation"), (col("count") / size(col("abb"))).alias("gpt")) \
    .distinct().groupBy(col("abbreviation")) \
    .agg(
        collect_list(col("player")).alias("players"), 
        size(collect_list(col("player"))).alias("size"), 
        sum(col("gpt")).alias("sum_gpt")) \
    .orderBy(col("size").desc(), col("sum_gpt").desc).limit(10).show(10, 100, True)